#### To convert the components into runnables, and make sure they all follow a standardized structure, we will need the concept of abstraction. We will create am abstract class --> Runnable, which will contain abstract methods.

#### All other component classes will inherit from the abstract Runnable class, and override the methods.

In [1]:
from abc import ABC,abstractmethod

class Runnable(ABC):
    
    @abstractmethod
    def invoke(input_data):
        pass

In [2]:
# now the llm class will inherit from the runnable class
# and it will override the invoke method
import random

class DummyLLM(Runnable):
    def __init__(self):
        print("LLM created.")
        
    def invoke(self,prompt):
        # this function does the same job as predict
        response_list = [
            "Delhi is the capital of India",
            "IPL is a cricket league in India.",
            "AI stands for Artificial Intelligence."
        ]
        
        # return a random response
        return {'response':random.choice(response_list)}

        
    def predict(self,prompt):
        response_list = [
            "Delhi is the capital of India",
            "IPL is a cricket league in India.",
            "AI stands for Artificial Intelligence."
        ]
        
        # return a random response
        return {'response':random.choice(response_list)}

In [3]:
# similarly prompt template will also inherit from Runnable class, and define the invoke method

class DummyPromptTemplate(Runnable):
    def __init__(self,template,input_variables):
        self.template = template
        self.input_variables = input_variables
        
    # define the invoke method, which does same job as format
    def invoke(self,input_dict):
        return self.template.format(**input_dict)
        
    def format(self,input_dict):
        return self.template.format(**input_dict)

So now, LLM and PromptTemplate both are runnables. And they follow the same structure.

Now you can connect these two classes to form chains, that can be as long as you want.

In [4]:
# create a runnableconnector class, which is also a runnable
# it takes a list of runnables, and create a chain
# o/p of first runnable goes into i/p of next runnable

class RunnableConnector(Runnable):
    def __init__(self,runnable_list):
        self.runnable_list = runnable_list
        
    def invoke(self,input_data):
        
        for runnable in self.runnable_list:
            input_data = runnable.invoke(input_data)
            
        return input_data             

In [5]:
# now if i send a list -> [prompt,llm]
# first the prompt gets invoked, and we get the prompt from its template
# then in the next iter, the llm gets invoke with prompt we got from previous step

prompt_template = DummyPromptTemplate(
    template="Write me a {length} poem on {topic}",
    input_variables=["length","topic"]
)

llm = DummyLLM()

# now pass them to chain
chain = RunnableConnector([prompt_template,llm])

# response
response = chain.invoke({'length':'short','topic':'India'})
print(response)

LLM created.
{'response': 'Delhi is the capital of India'}


In [8]:
# now we can also create a parser, and pass it as runnable without having to write any extra code in the chain class

class DummyStrOutputParser(Runnable):
    def __init__(self):
        pass
    
    def invoke(self,input_data):
        return input_data['response']

In [9]:
prompt_template = DummyPromptTemplate(
    template="Write me a {length} poem on {topic}",
    input_variables=["length","topic"]
)

llm = DummyLLM()

parser = DummyStrOutputParser()

# now pass them to chain
chain = RunnableConnector([prompt_template,llm,parser])

# response
response = chain.invoke({'length':'short','topic':'India'})
print(response)

LLM created.
IPL is a cricket league in India.


So this way, we can join as many runnables as we want.

#### Now we will join two chains. Since chains can itself be treated as runnables, so we will try joining htem now.

In [14]:
# chain1 -> create a joke
# chain2 -> explain the joke

template1 = DummyPromptTemplate(
    template="Create a joke on {topic}",
    input_variables=["topic"]
)

template2 = DummyPromptTemplate(
    template="Explain the following joke: \n {response}",
    input_variables=["response"]
)

joke_llm = DummyLLM()

joke_parser = DummyStrOutputParser()

# create chain1
chain1 = RunnableConnector([template1,joke_llm])

# create chain2
chain2 = RunnableConnector([template2,joke_llm,joke_parser])

# connect the two chains
final_chain = RunnableConnector([chain1,chain2])

# invoke the final chain
response = final_chain.invoke({'topic':'IPL'})

print(response)

LLM created.
IPL is a cricket league in India.
